# Langchain RAG with Mistral

This notebook is part of a medium article that I wrote about the Mistal 7B model. In this notebook I will show a simple example of how to use the Langchain RAG with the Mistral model. This can also be used with a series of other models from the LLM family. See the table below. 

First, intall the Mistal 7B model here:

## [Mistral 7B](https://ollama.ai/download)

## [Medium Article](Link to medium article)

In [15]:
!pip install langchain
!pip install gpt4all
!pip install chromadb
!pip install langchainhub

# Load data from the web
Let's load some data from the web. We will use the BBC article about the Japanese method of upcycling.

In [4]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.bbc.com/culture/article/20231013-the-300-year-old-japanese-method-of-upcyling")
data = loader.load()

# Split into chunks
We will split the document into chunks of 1500 characters with 100 characters overlap.

In [5]:
# Split into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

# Embed and store
We will embed the chunks using GPT4AllEmbeddings and store them in a Chroma vectorstore.

In [6]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings # We can also try Ollama embeddings
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=GPT4AllEmbeddings())

In [7]:
# RAG prompt
from langchain import hub
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

# Choose a model
We will use the Mistral model from the LLM family.

In [8]:
# LLM
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(model="mistral",
             verbose=True,
             callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [9]:
# QA chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

# Ask a question
Let's ask a question about the article.

In [10]:
from langchain.schema import LLMResult
from langchain.callbacks.base import BaseCallbackHandler

class GenerationStatisticsCallback(BaseCallbackHandler):
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(response.generations[0][0].generation_info)
        
callback_manager = CallbackManager([StreamingStdOutCallbackHandler(), GenerationStatisticsCallback()])

llm = Ollama(base_url="http://localhost:11434",
             model="mistral",
             verbose=True,
             callback_manager=callback_manager)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "What is this article about?"
result = qa_chain({"query": question})

The article is about sashiko, a basic form of stitching that is commonly used for denim repair but can also be applied to other woven fabrics. The technique involves pulling the yarn through the fabric from the front and up again, creating rows of stitches. Sashiko needles are longer than usual to allow multiple stitches to be made at once. The article discusses the benefits of sashiko workshops, including learning a versatile skill and meeting like-minded people.{'model': 'mistral', 'created_at': '2023-10-16T21:22:20.87282Z', 'response': '', 'done': True, 'context': [733, 16289, 28793, 10649, 28747, 733, 16289, 28793, 5275, 18741, 4060, 995, 460, 396, 13892, 354, 2996, 28733, 509, 28727, 2131, 9796, 28723, 5938, 272, 2296, 7769, 302, 17913, 286, 2758, 298, 4372, 272, 2996, 28723, 1047, 368, 949, 28742, 28707, 873, 272, 4372, 28725, 776, 1315, 369, 368, 949, 28742, 28707, 873, 28723, 5938, 1712, 23748, 7228, 304, 1840, 272, 4372, 3078, 864, 26364, 700, 18741, 4060, 28705, 13, 24994, 28

It appears that sashiko, a traditional Japanese mending technique, has become more popular in recent times. It is known for its aesthetic appeal, practicality, and versatility in various applications, including fashion, art, and home decor. The trend is driven by people's increasing focus on sustainability in the clothing industry and their desire to learn ancient crafts like sashiko. Sashiko workshops are being offered at venues such as museums and private tutors in both the UK and US.{'model': 'mistral', 'created_at': '2023-10-16T21:22:41.222468Z', 'response': '', 'done': True, 'context': [733, 16289, 28793, 10649, 28747, 733, 16289, 28793, 5275, 18741, 4060, 995, 460, 396, 13892, 354, 2996, 28733, 509, 28727, 2131, 9796, 28723, 5938, 272, 2296, 7769, 302, 17913, 286, 2758, 298, 4372, 272, 2996, 28723, 1047, 368, 949, 28742, 28707, 873, 272, 4372, 28725, 776, 1315, 369, 368, 949, 28742, 28707, 873, 28723, 5938, 1712, 23748, 7228, 304, 1840, 272, 4372, 3078, 864, 26364, 700, 18741, 40